In [ ]:
'''
Strategy: Weekly Buying based on Relative Strength [RS] and Hilega-Milega
Initial Screen Stocks from quarterly OHLC [Stocks Outperforming Index]
    - RS [compared with Nifty50] 
    - Trade Window: (RS > 0) to (RS < RS[-1])
    - Lookback 3 quarter
Weekly Screen from weekly OHLC
    - Buying HM-1 [Evaluates Price, Strength, Momentum and Volume]
    - WMA21 > WMA21[-1]
    - Positive RS and (RS[SMA5] > RS[-1][SMA5]) [Slope from Linear Regression should be tried]
Trade Parameter:
Entry Price
    - Type 1: Weekly Close
    - Type 2: 1/3 * [Entry Price - Stop Loss]
Stop Loss
    - Weekly Close < Swing Low Weekly     
Target
    - Type 1: 11 %
    - Type 2: 3 * [Entry Price - Stop Loss]
    
Input files: 
    - nse_total_list.csv
    - Datasets - quarterly and weekly OHLC (/raw_data/monthly_ohlc)
Output file: quarter_cup_handle_signals.csv
    - Stock_Name,Type,Entry_Type,Sig_Date,Cup_Date,Handle_Date,Buy_Price_1,Buy_Price_2
    - Type: [HM-1, HM-2, IN-1, IN-2]
    - Entry_Type : E1 [complete handle] - E2 [partial handle]
    - Buy_Price_1 : E1 trades
    - Buy_Price_2 : E2 trades

Idea:
HM signal   - Quarterly [HM-1, HM-2, IN-1, IN-2]
            - Entry_Price_1 [Entry_Price_2 from dataset]
Cup         - Monthly 
            - Init_1: Total Range < Entry_Price_2 
            - Sig_1: Close > Entry_Price_1
            - Entry_price_2 = low[Sig_1] (monthly close) 

Entry       - Type 1: Entry_Price_1
Target      - Type 1: 25 %


'''

In [ ]:
import sys
main_path = r'/Gokul/PYTHON-VS' 
sys.path.append(main_path) 

from scripts_Algo_GP import * 
from scipy.stats import linregress
import tradingview_indicators as ta
from pprint import pprint
import csv, math
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

In [ ]:
def rs_NIFTY(df,df_index,rs_length):
    # compute index s_ratio   
    df_index['i_ratio'] = df_index['close'] / df_index['close'].shift(rs_length)
    # compute return of stock
    df['s_ratio'] = df['close'] / df['close'].shift(rs_length)
    df['i_ratio'] = df_index['i_ratio']
    # Compute RS
    df['RS'] = ((df['s_ratio']/df['i_ratio'])-1).round(2)
    df = df.drop(columns=['s_ratio','i_ratio'])
    return df
#((152.96 /130.05)/(23644/22326))-1

In [ ]:
#def main():
'''------ Load Input Files ---------'''
#file_name = f"{main_path}/nse_list_2025_01.csv"
file_name = f"{main_path}/nse_total_list.csv"

file = open(file_name, "r")
list_stock = list(csv.reader(file, delimiter=","))
file.close()
#print(list_stock)

df_final = pd.DataFrame()
'''------ Input Parameters ---------'''
rs_qtr_length = 3
rs_week_length = 55
'''------ Load Index ---------'''
df_qtr_index = pd.read_csv(f"{main_path}/raw_data/quarter_ohlc/quarter_NIFTY.csv")
df_week_index = pd.read_csv(f"{main_path}/raw_data/weekly_ohlc/weekly_NIFTY.csv")
#print(df_qtr_index.tail(20))
'''------ Loop for each Stock---------'''
for stock_id in list_stock[2:]: 
    id_stock = stock_id[0]
    id_index= stock_id[1]
    qtr_stock = f"{main_path}/raw_data/quarter_ohlc/quarter_{id_stock[4:-3]}.csv"
    #week_stock = f"{main_path}/raw_data/weekly_ohlc/weekly_{id_stock[4:-3]}.csv"
    print(id_stock)
    #print(qtr_stock)
    #print(week_file)

    '''------ Initial Screen ---------'''
    
    if os.path.exists(qtr_stock):
        df_qtr = pd.read_csv(qtr_stock)
    else:
        continue
    df_qtr['datetime'] = pd.to_datetime(df_qtr['datetime']).dt.strftime('%Y-%m-%d')
    # Match the Size of the two DFs and reset Index number
    df_index = df_qtr_index.iloc[-len(df_qtr):].reset_index(drop=True)
    # Compute Relative Strength with Nifty50
    df_qtr = rs_NIFTY(df_qtr,df_index,rs_qtr_length)
    df_qtr = df_qtr[['datetime','RS']]
    df_qtr['start_RS'] = df_qtr['end_RS'] = 0  
    # Tradeable Intervals 
    mask_start= ((df_qtr['RS'] > 0) & (df_qtr['RS'].shift(1) <= 0)) \
        | ((df_qtr['RS'] > 0) & (df_qtr['RS'].shift(1) < df_qtr['RS']) \
        & (df_qtr['RS'].shift(1) < df_qtr['RS'].shift(2)))   
    df_qtr.loc[mask_start.shift(1).astype(bool).fillna(False), 'start_RS'] = 1
    df_qtr.at[0,'start_RS'] = 0
    mask_end = (df_qtr['RS'].shift(1) > df_qtr['RS']) \
        & (df_qtr['RS'].shift(1) > df_qtr['RS'].shift(2)) \
        & (df_qtr['RS'].shift(1) > 0)
    df_qtr.loc[mask_end, 'end_RS'] = -1
    # Add trading interval to df
    df_qtr.loc[(df_qtr['start_RS'] == 1), 'start_interval'] = df_qtr.loc[(df_qtr['start_RS'] == 1), 'datetime']
    df_qtr.loc[(df_qtr['end_RS'] == -1), 'end_interval'] = pd.to_datetime(df_qtr.loc[(df_qtr['end_RS'] == -1), 'datetime']) + pd.offsets.QuarterEnd(0)
    df_qtr['end_interval'] = df_qtr['end_interval'].dt.strftime('%Y-%m-%d')
    # Create an array of tradeable interval Dates
    start_array = df_qtr['start_interval'].dropna().tolist()
    end_array = df_qtr['end_interval'].dropna().tolist()
    #print(start_array)
    #print(end_array)
    del df_index

    '''------ Weekly Screen ---------'''
    df_week_index = df_week_index
    week_stock = f"{main_path}/raw_data/weekly_ohlc/weekly_{id_stock[4:-3]}.csv"
    df_week_stock = pd.read_csv(week_stock)
    df_qtr['datetime'] = pd.to_datetime(df_qtr['datetime']).dt.strftime('%Y-%m-%d')
    df_week_stock['sig_week'] = 0 
    # Match the Size of the two DFs and reset Index number
    df_index = df_week_index.iloc[-len(df_week_stock):].reset_index(drop=True)
    #print(df_index.tail(5))
    ''' ___ COmpute Percentage Change ___ '''
    df_week_stock['change'] = (((df_week_stock['close']-df_week_stock['close'].shift(1))/df_week_stock['close'])*100).round(1)
    '''___ Compute Relative Strength with Nifty50 and Moving Average ___'''
    df_week_stock = rs_NIFTY(df_week_stock, df_index, rs_week_length)
    df_week_stock['SMA_RS'] = df_week_stock['RS'].rolling(window=5).mean().round(2)
    ''' ___ Hil Mil Mask & Moving Average [WMA]___'''
    len_ema = 3
    df_week_stock['RSI9'] = ta.RSI(df_week_stock['close'],9).round(2)
    df_week_stock['EMA3'] = df_week_stock["RSI9"].ewm(span=3, min_periods=3).mean().round(2)
    df_week_stock['WMA21'] = IndicatorsTV().wma(df_week_stock['RSI9'],21).round(2)
    df_week_stock['HM'] = 0
    # HM 1 - Positive
    mask_hm = (
        (df_week_stock['RSI9'] > df_week_stock['WMA21']) 
        & (df_week_stock['RSI9'] > df_week_stock['EMA3']) 
        & (df_week_stock['WMA21'].shift(1) > df_week_stock['RSI9'].shift(1)) 
    )
    #print(mask_hm)
    df_week_stock.loc[mask_hm,'HM'] = 1
    df_week_stock['SMA_WMA'] = df_week_stock['WMA21'].rolling(window=3).mean().round(2)

    ''' ___ Signal Generation - Masks ___ '''
    mask_weekly = (
        (df_week_stock['HM'] == 1)
        & (df_week_stock['WMA21'] > df_week_stock['SMA_WMA'].shift(1))
        & (df_week_stock['RS'] > 0)
        & (df_week_stock['RS'] > df_week_stock['SMA_RS'])
        & (df_week_stock['SMA_RS'] > df_week_stock['SMA_RS'].shift(1))
    )
    df_week_stock.loc[mask_weekly,'sig_week'] = 1
    df_week_stock = df_week_stock.drop(columns=['RS','WMA21','EMA3','HM','SMA_RS','SMA_WMA','RSI9'])
    #print(df_week_stock.head(60))

    '''------ Entry screen ---------'''
    df_entry = pd.DataFrame()
    for start, end in zip(start_array, end_array):
        df_trim_week = df_week_stock[(df_week_stock['datetime'] >= start) & (df_week_stock['datetime'] <= end)].copy()
        df_trim_week['entry_deadline'] = end
        #print(df_trim_week)
        df_entry = pd.concat([df_entry, df_trim_week[(df_trim_week['sig_week'] == 1)]], ignore_index=False)
        #break
    df_entry = df_entry.reset_index()
    #df_entry = df_entry.reset_index(drop=True)

    '''------ Trade Module ---------'''
    df_traded = df_entry
    for traded in range(len(df_entry)):
        
        '''___ Entry ___'''
        entry_price = df_entry.iloc[traded]['close']
        df_tmp = df_week_stock.iloc[df_entry.iloc[traded]['index']+1:]
        first_instance = df_tmp[df_tmp['low'] < entry_price].head(1)
        #df_traded.loc[traded, 'entry_index'] = int(first_instance.index[0])
        #df_traded['entry_index'] = df_traded['entry_index'].astype(int)
        # Ensure first_instance is not empty before accessing its index
        if not first_instance.empty:
            df_traded.loc[traded, 'entry_index'] = int(first_instance.index[0])
            df_traded.loc[traded, 'entry_week'] = first_instance['datetime'].iloc[0]

        else:
            df_traded.loc[traded, 'entry_index'] = np.nan  # Assign NaN if no match is found
            df_traded.loc[traded, 'entry_week'] = df_traded.loc[traded,'datetime']
        # Fill NaN values with a default integer before conversion (choose a meaningful value)
        df_traded['entry_index'] = df_traded['entry_index'].fillna(-1).astype(int)
        #df_traded['entry_week'] = df_traded['entry_week'].fillna(-1).astype(int)

        #df_traded.loc[traded, 'entry_week'] = first_instance['datetime'].iloc[0]
        del df_tmp, first_instance
        '''___ Target ___'''
        tgt_perc = 11
        tgt_price = (entry_price + (tgt_perc*0.01*entry_price)).round(1)
        #print(tgt_price)
        df_tmp = df_week_stock.iloc[df_entry.iloc[traded]['entry_index']:]
        first_instance = df_tmp[df_tmp['high'] > tgt_price].head(1)
        if not first_instance.empty:
            df_traded.loc[traded, 'exit_week'] = first_instance['datetime'].iloc[0]
            df_traded.loc[traded, 'profit'] = tgt_perc
        else:
            df_traded.loc[traded, 'exit_week'] = df_week_stock['datetime'].iloc[-1]
            df_traded.loc[traded, 'profit'] = (((df_week_stock['close'].iloc[-1]-entry_price)*100)/entry_price).round(1)
        df_traded['duration'] = pd.to_datetime(df_traded['exit_week']) -pd.to_datetime(df_traded['entry_week'])
        df_traded['stock'] = id_stock
        del df_tmp, first_instance
        # Remove rows where 'traded_idx' equals -1
        
    if df_traded.empty:
        continue
    df_traded = df_traded[df_traded['entry_index'] != -1]
    #print(df_traded)
    df_tmp = df_traded[['stock','datetime','entry_week','exit_week','profit','duration','change']]
    #print(df_tmp)
    if df_final.empty:
        df_final = df_tmp
    else:
        df_final = pd.concat([df_final , df_tmp])
        df_final.reset_index(drop=True, inplace=True)      
        #print(df_final)
df_final.to_csv('first_trail.csv', index=False)


            
#if __name__ == '__main__':
#    main()
